In [1]:
import json
import os
with open('./pleiades-places.json','r', encoding="utf8") as json_file, open('./pleiades_jsonline', 'a') as f1:
    f = json.load(json_file)
    for json_place in f["@graph"]:
        f1.write(json.dumps(json_place) + "\n")

In [1]:
import networkx as nx
import pandas as pd
df = pd.read_json(r'./pleiades_jsonline',lines=True)

In [25]:
df.head()

,features,contributors,locations,connections,references,names,id,subject,title,provenance,...,reprPoint,placeTypes,bbox,connectsWith,rights,created,uri,creators,@type,history


In [3]:
df_filtered = df.drop(columns=['provenance', 'review_state', 'rights', 'reprPoint','type', 'created', 'creators', 'references', 'contributors', 'history'])
df.head()
df.shape

(37151, 25)

In [4]:
df_num_without_conn = df_filtered[df_filtered['connectsWith'].map(lambda d: len(d)) > 0]
df_num_without_conn["names"] = df_num_without_conn['names'].apply(lambda x: [i['romanized'] for i in x])
df_num_without_conn.head()["names"].values.tolist()

C:\Users\Jeremy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[[],
 [],
 ['Contestania', 'Contestani'],
 ['Dianium', 'Hemeroskopeion', 'Dénia, Denia'],
 ['Ebusus']]

In [5]:
df_num_without_conn.head()["names"].values.tolist()[2]

['Contestania', 'Contestani']

In [6]:
len(df_num_without_conn.index)

7237

In [7]:
def parse_id(s):
    return s[s.rfind("/")+1:]

In [42]:
G = nx.Graph()
for index, row in df_num_without_conn.iterrows():
    G.add_node(str(row['id']), title=row['title'])
    for c in row['connectsWith']:
        if "places" in c:
            G.add_edge(str(row['id']), parse_id(c))
len(G.nodes.data())

8116

In [12]:
print(len(G.edges.keys()))
print(len(G.nodes.keys()))

8406

In [13]:
G.remove_nodes_from(list(nx.isolates(G)))

In [14]:
print(len(G.edges.keys()))
print(len(G.nodes.keys()))

8406
8116


In [16]:
pr = nx.pagerank(G)
pr_results = dict(sorted(pr.items(), key=lambda x: x[1], reverse=True)[:5])
print(pr_results)

{'462492': 0.018460249378171586, '472014': 0.014551570445262835, '91358': 0.013730000606534417, '423025': 0.011384123823953783, '707498': 0.00772340418048712}


In [17]:
for i in pr_results:
    print(G.nodes[i])
    print()

{'title': 'Sicilia (island)'}

{'title': 'Sardinia (island)'}

{'title': "Hadrian's Wall"}

{'title': 'Roma'}

{'title': 'Cyprus (island)'}



In [45]:
from networkx.algorithms import community
communities_generator = community.label_propagation_communities(G)

In [46]:
communities = [x for x in communities_generator]

In [47]:
len(communities)

1668

In [29]:
from networkx.readwrite import json_graph
import json
data1 = json_graph.node_link_data(G)
nodes = []
for d in data1['nodes']:
    if len(G.nodes[d['id']]):
        d["title"] = G.nodes[d['id']]['title']
    else:
        d["title"] = d['id']
    nodes.append(d)
data1['nodes'] = nodes


In [30]:
with open('graph.json', 'w') as f:
    json.dump(data1, f)